In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import random
import pprint as pp
pprint = pp.PrettyPrinter(indent=4).pprint

In this document, we'll use the Lagrangian formulation of classical mechanics to simulate physical systems.

The code works like this: You specify an initial configuration of the system and a terminal configuration of the system, and a Lagrangian, and a simulation fidelity, and we find a path through configuration-space (in segments according to the fidelity) that approximately minimizes the Lagrangian.

(If you instead want to maximize the Lagrangian, you'll have to change the optimizer. If you instead want to stabilize the action at an inflection point... then good luck to you.) 

In [183]:
dim = 2             # Dimensionality of the space
start_time = 0.0    # Time to begin simulation
end_time = 1.0      # Time to end simulation
fidelity = 10        # Frames per second

num_particles = 3   # Number of particles in the space
masses = [1.0, 5.0, 10.0]
initial_config = [
    [100.0, -100.0],
    [-50, 50],
    [0, 0],
]
final_config = [
    [0.0, 0.0],
    [-50, 50],
    [-100, 100],
]

def V(ms, qs):
    # return 100 * ms * qs[:,1]  # Parabolas
    return 10 * tf.reduce_sum(ms * tf.reduce_sum(qs * qs, 1))  # Harmonic oscillators
    # return 0  # Straight lines

def T(ms, qdots):
    ke = 0.5 * tf.reduce_sum(ms * tf.reduce_sum(qdots * qdots, 1))
    ke = tf.Print(ke, [ke, ms, qdots], message='kinetic energy (from ms, qs, qdots) is ')
    return ke

def L(ms, qs, qdots):
    return T(ms, qdots) - V(ms, qs)

In [184]:
num_steps = 1000
total_time = end_time - start_time
num_frames = int(total_time * fidelity)
delta_t = 1.0 / fidelity

assert len(masses) == num_particles
assert len(initial_config) == num_particles
assert len(final_config) == num_particles
assert all(len(q) == dim for q in initial_config)
assert all(len(q) == dim for q in final_config)

graph = tf.Graph()
with graph.as_default():
    # You specify these.
    mass_tensor = tf.constant(masses, name='masess')
    initial_config_tensor = tf.constant(initial_config, name='initial_config')
    final_config_tensor = tf.constant(final_config, name='final_config')
    
    # We approximate these
    intermediary_configs = tf.Variable(tf.zeros([num_frames, num_particles, dim]))
    
    # like this.
    def positions(i):
        assert 0 <= i <= num_frames
        return initial_config_tensor if i == 0 else intermediary_configs[i-1]
        
    def velocities(i):
        assert 0 <= i <= num_frames
        now = intermediary_configs[i-1] if i > 0 else initial_config_tensor
        next = intermediary_configs[i] if i < num_frames else final_config_tensor
        return (next - now) / delta_t
    
    def lagrangian(i):
        return L(mass_tensor, positions(i), velocities(i))
    
    action = tf.reduce_sum([lagrangian(i) for i in range(num_frames + 1)])
    
    # Optimization
    optimizer = tf.train.GradientDescentOptimizer(0.0005)
    step = optimizer.minimize(action)

In [185]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for i in range(num_steps):
        if i % 100 == 0:
            frames = intermediary_configs.eval()
            print('Frame sample at step %d:' % i)
            pprint(frames)
        step.run()
    print('Finished! The final trajectory looks like this:')
    init, intermed, final = session.run([initial_config_tensor, intermediary_configs, final_config_tensor])
    pprint(init)
    pprint(intermed)
    pprint(final)

Initialized
Frame sample at step 0:
array([[[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]]], dtype=float32)
Frame sample at step 100:
array([[[  9.60524826e+01,  -9.60524826e+01],
        [ -2.55482056e+02,   2.55482056e+02],
        [ -3.87818164e+03,   3.87818164e+03]],

       [[  7.94342041e+01,  -7.94342041e+01],
        [ -4.36928711e+02,   4.36928711e+02],
        [ -7.44750293e+03,   7.44750293e+03]],

       [[  5.81270218e+